In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
%%capture
%pwd

In [ ]:
%%capture
cd ..

In [ ]:
from random import sample 
from src.funcs.utils import *

In [ ]:
# api keys, institutional tokens and Pharmapendium module urls in .env file
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv('PP_API_KEY')
insttoken=os.getenv('PP_INSTTOKEN')


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Pharmapendium Safety Query

#### Query for Available Data Fields

In [ ]:
# set parameters for api call
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_safety_fields = os.getenv('URL_SAFETY_FIELDS')

In [ ]:
# use utility function to retrieve available data fields
df_fields=get_data_fields(url_safety_fields, parameters)

In [ ]:
df_fields

#### Query for Available Facet Fields

In [ ]:
# set parameters for api call
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_facets = os.getenv('URL_SAFETY_FACET_FIELDS')

In [ ]:
# utility function to retrieve available facets fields
df_facets=get_facet_fields(url_facets, parameters)

In [ ]:
df_facets

#### Query for All Clinical Drugs

In [ ]:
# set parameters for api call
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters["limitation.count"] = 0    #set to 0 when using facets
parameters["facets"]="drugs"
parameters['specie']='human'
parameters['leaf']=True
url_safety=os.getenv('URL_SAFETY_SEARCH')

In [ ]:
# test status code of api call (200 is successful api call)
with requests.Session() as s:
    r = s.get(url_safety, params=parameters)
r.status_code 

In [ ]:
# utility function to get all drugs
drugs_df=get_drugs(url_safety, parameters)

In [ ]:
drugs_df.info(); drugs_df.head(10)

In [ ]:
drugs_df.to_pickle('./data/raw/clinical_drugs.pkl')

#### Query for Pre-Marketing Adverse Events

In [ ]:
drugs_df=pd.read_pickle('./data/raw/clinical_drugs.pkl')

In [ ]:
drugs_df.info(); drugs_df.head()

In [ ]:
drug_list=list(drugs_df.drug)
len(drug_list)
drug_sample=drug_list[:5]  #take the first 5 drugs in list for demonstration

In [ ]:
# set parameters for api call
parameters = dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count']=0     #set to 0 when using facets
parameters['species']='human'
parameters['leaf']=True
parameters['facets'] = 'effects'
url_safety = os.getenv('URL_SAFETY_SEARCH')

In [ ]:
# utilitiy function to retrieve all premarketying adverse events 
df_aes=get_drug_ae(url_safety, drug_sample, parameters)

In [ ]:
df_aes.info(); df_aes.head()

### Pharmapendium PK Query

In [ ]:
# set parameters for api call
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters["limitation.count"] = 0   #set to 0 when using facets
parameters["facets"]="drugs"
parameters['specie']='human'
parameters['leaf']=True
url_pk=os.getenv('URL_PK_SEARCH')

In [ ]:
# check status code of api call (200 is a successful api call)
with requests.Session() as s:
    r = s.get(url_safety, params=parameters)
r.status_code 

In [ ]:
# utility function to retrieve drugs in module
pk_drugs_df=get_drugs(url_safety, parameters)

In [ ]:
pk_drugs_df.info(); pk_drugs_df.head()

#### Query for Available Fields

In [ ]:
# set parameters for api call
parameters = dict()

parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_pk=os.getenv('URL_PK_FIELDS')

In [ ]:
# utility function to retrieve data fields
df_fields=get_data_fields(url_pk, parameters)

In [ ]:
df_fields

#### Query for all PK parameters

In [ ]:
# set parmeters for api call
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count']=0 
parameters['species']='human'
parameters['facets']='parameters'
parameters['leaf']=True
url_pk=os.getenv('URL_PK_SEARCH')

In [ ]:
# utilitly function to retrieve available PK parameters
df_params=get_pk_params(url_pk, parameters)


In [ ]:
df_params.info(); df_params

#### Query for PK Data

In [ ]:
drugs_df=pd.read_pickle('./data/raw/clinical_drugs.pkl')

In [ ]:
drugs_df.head()

In [ ]:
# set PK parameters and fields of interest

pk_param_list=['AUC', 'T1/2', 'Cmax', 'Vd', 'CL', 'Plasma protein binding', 'Bioavailability']

field_list=['drug', 'parameter', 'sampleSize', 'sex', 'dose', 'duration', 'route', 'comments','valueNormalized', 'unitNormalized', 'valueOriginal']

drugList=list(drugs_df.drug)

drug_sample=drugList[:5]


In [ ]:
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters["limitation.count"] = 0
parameters["facets"]="drugs"
parameters['species']='human'
# parameters['leaf']=True
url_pk=os.getenv('URL_PK_SEARCH')

In [ ]:
# sanity check- test counts vs. GUI
hit_count_list=[]
url=os.getenv('URL_PK_SEARCH')
with requests.Session() as s:
    for drug in tqdm.tqdm(drug_sample):
        parameters["drugs"] = drug
        parameters['limitation.count']=0
        parameters['sources']= 'FDA Approval Packages', 'EMA approval documents'
        r = s.get(url, params=parameters).json()
        hits = r['data']['countTotal']
        hit_count_list.append(int(hits))
        

In [ ]:
hit_count_list

In [ ]:
# set parameters for api call
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['species']='human'
url_pk=os.getenv('URL_PK_SEARCH')

In [ ]:
# choose PK parameters and data fields of interest
pk_param_list=['AUC', 'T1/2', 'Cmax']
field_list=['drug', 'parameter', 'sampleSize', 'sex', 'dose']
drug_sample=drugList[:5] #select first 5 drugs for demonstration

In [ ]:
# utility function to retrieve PK data
df=get_PK_data(url_pk, drug_sample, field_list,  pk_param_list,  parameters)

In [ ]:
df.info(); df.head()

In [ ]:
df.to_csv('./data/raw/pp_pk_drugs.csv', index=False)

### Pharmapendium FAERS Query

#### Query for all data fields

In [ ]:
# set parameters for api call
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_faers_fields=os.getenv('URL_FAERS_FIELDS')

In [ ]:
# utility function to retrieve available data fields
df_fields=get_data_fields(url_faers_fields, parameters)

In [ ]:
df_fields

#### Query for Available Facets Fields

In [ ]:
# set parameters for api call
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_faers_facets=os.getenv('URL_FAERS_listFacets')

In [ ]:
# utility function to retrieve available facets fields
df_facets=get_facet_fields(url_faers_facets, parameters)

In [ ]:
df_facets

#### Query for All Drugs in FAERS

In [ ]:
# set parameters for api call
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count'] = 0
parameters['facets']="drugs"
parameters['leaf']=True
url_faers=os.getenv('URL_FAERS')

In [ ]:
# check status code of api call (200 is a successful api call)
with requests.Session() as s:
    r = s.get(url_faers, params=parameters)
r.status_code 

In [ ]:
# utility function to retrieve available drugs
drugs_df=get_drugs(url_faers, parameters)

In [ ]:
drugs_df.info(); drugs_df.head()

In [ ]:
drugs_df.to_pickle('./data/raw/faers_drugs.pkl')

#### Query for Post-marketing Drug AEs

In [ ]:
drugs_df=pd.read_pickle('./data/raw/faers_drugs.pkl')

In [ ]:
drug_list=list(drugs_df.drug)
len(drug_list)
drug_sample[:5]

In [ ]:
# set parameters for api call
parameters=dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
parameters['limitation.count'] = 0  
parameters['facets']="effects"
parameters['leaf']=True

url_faers=os.getenv('URL_FAERS')

In [ ]:
# utility function to retrieve drug AEs from FAERS
df_aes_faers=get_drug_aes_faers(url_faers, drug_sample, parameters)

In [ ]:
df_aes_faers.info(); df_aes_faers.head()

In [ ]:
df_aes_faers.ae.value_counts()

### Chemistry Query

#### Query for Available Data Fields

In [ ]:
# retrieve URLs from .env file
url_chem_fields=os.getenv('URL_CHEM_FIELDS')
url_chem_facets_fields=os.getenv('URL_CHEM_FACET_FIELDS')
url_chem_post_search=os.getenv('URL_CHEM_SEARCH')

In [ ]:
# set parameters for api call
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_chem_fields=os.getenv('URL_CHEM_FIELDS')

In [ ]:
# utility function for retrieving available data fields
df_fields=get_data_fields(url_chem_fields, parameters)

In [ ]:
df_fields

#### Query for Available Facet Fields

In [ ]:
# set parameters for api call
parameters=dict()
parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_chem_facets=os.getenv('URL_CHEM_FACET_FIELDS')

In [ ]:
# utility function to retrieve available facets fields
df_facets=get_facet_fields(url_chem_facets, parameters)

In [ ]:
df_facets

#### Chem Structure Search

In [ ]:
# Note, double backslashes required.
data=dict()
data='{"structure":"\\n  Ketcher 04282111472D 1   1.00000     0.00000     0\\n\\n 13 13  0     0  0            999 V2000\\n  -12.8250    0.5500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -11.9590    0.0500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -11.9590   -0.9500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -12.8250   -1.4500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -13.6910   -0.9500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -13.6910    0.0500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -11.0929    0.5500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -10.2269    0.0500    0.0000 O   0  0  0  0  0  0  0        0  0  0\\n  -11.0929    1.5500    0.0000 O   0  0  0  0  0  0  0        0  0  0\\n  -11.0929   -1.4500    0.0000 O   0  0  0  0  0  0  0        0  0  0\\n  -11.0929   -2.4500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -10.2269   -2.9500    0.0000 C   0  0  0  0  0  0  0        0  0  0\\n  -11.9590   -2.9500    0.0000 O   0  0  0  0  0  0  0        0  0  0\\n  1  2  1  0     0  0\\n  2  3  2  0     0  0\\n  3  4  1  0     0  0\\n  4  5  2  0     0  0\\n  5  6  1  0     0  0\\n  6  1  2  0     0  0\\n  2  7  1  0     0  0\\n  7  8  1  0     0  0\\n  7  9  2  0     0  0\\n  3 10  1  0     0  0\\n 10 11  1  0     0  0\\n 11 12  1  0     0  0\\n 11 13  2  0     0  0\\nM  END\\n", "type":"Substructure", "drugs":["Analgesics"]}'

In [ ]:
# set parameters for an api post call
headers=dict()

headers['X-ELS-APIKey']=api_key
headers['X-ELS-Insttoken']=insttoken
headers['Content-Type']='application/json'
headers['Accept']='application/json'
url_chem_post_search=os.getenv('URL_CHEM_SEARCH')


In [ ]:
headers

In [ ]:
# perform api call and check status code
r = requests.post(url_chem_post_search, headers=headers, data=data )
r.status_code

In [ ]:
# inspect output
print(r.text)

### Efficacy

#### Query for Available Data Fields

In [ ]:
# set parameters for api call
parameters = dict()

parameters['apiKey']=api_key
parameters['insttoken']=insttoken
url_efficacy_fields = os.getenv('URL_EFFICACY_FIELDS')

In [ ]:
# utility function to retrieve available data fields
df_fields=get_data_fields(url_efficacy_fields, parameters)

In [ ]:
df_fields

#### Query for Available Facet Fields

In [ ]:
# set parameters for api call
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
url_efficacy_facets = os.getenv('URL_EFFICACY_FACET_FIELDS')


In [ ]:
# utility function to retrieve available facets fields
df_facets=get_facet_fields(url_efficacy_facets, parameters)

In [ ]:
df_facets

#### Query for Indication

In [ ]:
# set parameters for api call
parameters = dict()
parameters['apiKey'] = api_key
parameters['insttoken'] = insttoken
parameters['drugs']='Abacavir Sulfate'
parameters['limitation.count']=0
parameters['species']='human'
parameters['facets'] = ['indications', 'species']
parameters['leaf']=True
url_efficacy_search=os.getenv('URL_EFFICACY_SEARCH')

In [ ]:
# api call
with requests.Session() as s:
        r = s.get(url_efficacy_search, params=parameters).json()

In [ ]:
# check output
r['data']
r['facets']